## 확률론적 언어 모형
m개의 단어 열(word sequence)이 주어졌을 때 문장으로써 성립될 확률을 출력함으로써 이 단어 열이 실제로 현실에서 사용될 수 있는 문장인지 판별하는 모형

다음과 같이 계산
$$\begin{eqnarray}
P(w_1, w_2, \ldots, w_m) &=& P(w_1, w_2, \ldots, w_{m-1}) \cdot P(w_m\;|\; w_1, w_2, \ldots, w_{m-1}) \\
&=& P(w_1, w_2, \ldots, w_{m-2}) \cdot P(w_{m-1}\;|\; w_1, w_2, \ldots, w_{m-2}) \cdot P(w_m\;|\; w_1, w_2, \ldots, w_{m-1}) \\
&=& P(w_1) \cdot P(w_2 \;|\; w_1) \cdot  P(w_3 \;|\; w_1, w_2) P(w_4 \;|\; w_1, w_2, w_3) \cdots P(w_m\;|\; w_1, w_2, \ldots, w_{m-1})
\end{eqnarray}$$


1. $P(w_m\;|\; w_1, w_2, \ldots, w_{m-1})$ 은 
1. 지금까지 $ w_1, w_2, \ldots, w_{m-1}$라는 단어 열 **(문맥, context)** 이 나왔을 때
1. 다음 단어로 $w_m$이 나올 조건부 확률

조건부 확률을 모형화하는 방법에 따라
1. Unigram Model
1. Bigram Model
1. N-gram Model 

로 나뉘어 진다.

## Unigram Model
모든 단어의 활용이 서로 완전히 독립이라고 가정
- 단어 열의 확률 = 각 단어 확률의 곱


## Bigram Model (Markov Model)
단어의 활용이 `바로 전 단어`에만 의존한다고 가정
$$P(w_1, w_2, \ldots, w_m) = P(w_1) \prod_{i=2}^{m} P(w_{i}\;|\; w_{i-1})$$


## N-gram Model
바로 전 N개의 단어에 의존한다고 가정
$$P(w_1, w_2, \ldots, w_m) = P(w_1) \prod_{i=n}^{m} P(w_{i}\;|\; w_{i-1}, \ldots, w_{i-n})$$

## 확률 추정 방법
- Bigram model

    1. 모든 문장에 문장의 시작과 끝을 나타내는 특별 토큰을 추가. 
    1. 전체 문장의 확률은 다음과 같은 조건부 확률의 곱
    
    $$P(\text{SS I am a boy SE}) = P(\text{I}\;|\; \text{SS}) \cdot P(\text{am}\;|\; \text{I}) \cdot P(\text{a}\;|\; \text{am}) \cdot P(\text{boy}\;|\; \text{a}) \cdot P(\text{SE}\;|\; \text{boy})$$
    
    1. 다음과 같이 추정
    
    $$P(w_{i}\;|\; w_{i-1}) = \dfrac{C(w_{i}, w_{i-1})}{C(w_{i-1})} = \dfrac{N_{real}}{N_{chance}}$$
    
    단어의 실제 등장 횟수 / 단어가 나올수 있는 기회(전체 유니그램이 나타나는 횟수)

In [1]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to /home/mk/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /home/mk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.corpus import movie_reviews

sentences = []
for s in movie_reviews.sents():
    s.insert(0, "SS")
    s.append("SE")
    if len(s) > 4:
        sentences.append(s)

In [3]:
sentences[1]

['SS', 'they', 'get', 'into', 'an', 'accident', '.', 'SE']

In [4]:
from collections import Counter

def calculate_bigram(sentences):
    bigram = {}
    for s in sentences:
        context = "SS"
        for i, w in enumerate(s[1:]):
            if context not in bigram:
                bigram[context] = Counter()
            if bigram[context][w] == 0:
                bigram[context][w] = 1
            bigram[context][w] += 1
            context = w
    for context in bigram.keys():
        total = sum(bigram[context].values())
        for w in bigram[context]:
            bigram[context][w] /= total
    return bigram

In [5]:
bigram = calculate_bigram(sentences)

문장 처음에 올 수 있는 단어들

In [6]:
bigram["SS"].most_common(10)

[('the', 0.11231263830320237),
 ('it', 0.043575076893101194),
 ('i', 0.03379121261464379),
 ('but', 0.02523207103391647),
 ('and', 0.024160438673402642),
 ('he', 0.023269731256871668),
 ('in', 0.023102723616272112),
 ('this', 0.022963550582439148),
 ('there', 0.0180507424881355),
 ('as', 0.013249272820898222)]

In [18]:
bigram["i"].most_common(10)

[("'", 0.14362771020624007),
 ('was', 0.053622421998942356),
 ('can', 0.03722897937599154),
 ('have', 0.035007932310946586),
 ('don', 0.02929666842940243),
 ('think', 0.028027498677948175),
 ('had', 0.02147012162876785),
 ('would', 0.021152829190904283),
 ('know', 0.018826017979904814),
 ('didn', 0.01850872554204125)]

In [26]:
bigram["."]

Counter({'"': 0.02922949299760894,
         "'": 0.0010735373054213634,
         "''": 6.506286699523415e-05,
         ')': 0.00821418695814831,
         'SE': 0.9612387969875893,
         ']': 0.0001789228842368939})

'he' 단어 뒤에 'is'가 나올 확률

In [24]:
bigram["he"]["is"]

0.09349147812219676

### 문장의 확률 

In [27]:
def sentence_score(s):
    p = 0.0
    for i in range(len(s) - 1):
        c = s[i]
        w = s[i + 1]
        p += np.log(bigram[c][w] + np.finfo(float).eps)
    return np.exp(p)

In [28]:
test_sentence = ["i", "am", "a", "boy", "."]
sentence_score(test_sentence)

3.288036438066686e-08

In [29]:
test_sentence = ["i", "is", "boy", "a", "."]
sentence_score(test_sentence)

1.9683389110380156e-38

## 문장 생성

In [30]:
def generate_sentence(seed=None):
    if seed is not None:
        np.random.seed(seed)
    c = "SS"
    sentence = []
    while True:
        if c not in bigram:
            break
        words, probs = zip(*[(k, v) for k, v in bigram[c].items()])
        idx = np.argmax(np.random.multinomial(1, probs, (1,)))
        w = words[idx]
        
        if w == "SE":
            break
        elif w in ["i", "ii", "iii"]:
            w2 = w.upper()
        elif w in ["mr", "luc", "i", "robin", "williams", "cindy", "crawford"]:
            w2 = w.title()
        else:
            w2 = w
        
        if c == "SS":
            sentence.append(w2.title())
        elif c in ["`", "\"", "'", "("]:
            sentence.append(w2)
        elif w in ["'", ".", ",", ")", ":", ";", "?"]:
            sentence.append(w2)
        else:
            sentence.append(" " + w2)
            
        c = w
    return "".join(sentence)

In [31]:
generate_sentence(82)

'Alexandre dumas may suspect he at being can be honest here goes awol, but he trusts affleck - see this documentary.'

활용
- spell correction 철자, 문법 교정
- speech recognition 음성 인식 
- machine translation 자동 번역
- Summarization 자동 요약
- Question-Answering 챗봇

---